In [85]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import re
import os

In [47]:
# Load the dataset
dataset = load_dataset("imruqays/quran-classical-arabic-english-parallel-texts")
df = dataset['train'].to_pandas()

In [194]:
display(df.sample())
display(df.info())

,arabic-uthmanic,arabic-imlaei,en-ahmedali,en-ahmedraza,en-arberry,en-asad,en-daryabadi,en-hilali,en-itani,en-maududi,en-mubarakpuri,en-pickthall,en-qarai,en-qaribullah,en-sahih,en-sarwar,en-shakir,en-wahiduddi,en-yusufali
5230,وَإِذْ أَسَرَّ ٱلنَّبِىُّ إِلَىٰ بَعْضِ أَزْوَ...,وَإِذ أَسَرَّ النَّبِيُّ إِلىٰ بَعضِ أَزواجِهِ...,When the Prophet told one of his wives somethi...,And when the Holy Prophet confided a matter to...,And when the Prophet confided to one of his wi...,And lo! [It so happened that] the Prophet told...,And recall what time the Prophet confided a st...,And (remember) when the Prophet (SAW) disclose...,The Prophet told something in confidence to on...,The Prophet confided something to one of his w...,And (remember) when the Prophet disclosed a ma...,When the Prophet confided a fact unto one of h...,When the Prophet confided a matter to one of h...,The Prophet confided a certain matter to one o...,And [remember] when the Prophet confided to on...,The Prophet told a secret to one of his wives ...,And when the prophet secretly communicated a p...,The Prophet once told one of his wives somethi...,When the Prophet disclosed a matter in confide...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6235 entries, 0 to 6234
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   arabic-uthmanic  6235 non-null   object
 1   arabic-imlaei    6235 non-null   object
 2   en-ahmedali      6235 non-null   object
 3   en-ahmedraza     6235 non-null   object
 4   en-arberry       6235 non-null   object
 5   en-asad          6235 non-null   object
 6   en-daryabadi     6235 non-null   object
 7   en-hilali        6235 non-null   object
 8   en-itani         6235 non-null   object
 9   en-maududi       6235 non-null   object
 10  en-mubarakpuri   6235 non-null   object
 11  en-pickthall     6235 non-null   object
 12  en-qarai         6235 non-null   object
 13  en-qaribullah    6235 non-null   object
 14  en-sahih         6235 non-null   object
 15  en-sarwar        6235 non-null   object
 16  en-shakir        6235 non-null   object
 17  en-wahiduddi     6235 non-null   

None

In [195]:
data = df.copy()

In [196]:
# Remove text between parentheses (Explanatory text which is not originally written in arabic)

data = data.map(lambda text: re.sub(r'\([^)]*\)|\[[^]]*\]', '', text))
# Combine dashed verses to complete the sentences.

connection_signs = ['-', '—', ':', ';', ',']

for index, row in data.iterrows():
    
    total_connected_sentences = sum(row[column].endswith(connection_sign) 
                                    for column in data.columns for connection_sign in connection_signs)

    # If there is at least 2 translations that say the Ayah is connected with the next, then connect them.
    if total_connected_sentences > 3:
        for column in data.columns:
            data.at[index + 1, column] = f"{data.at[index, column]} {data.at[index + 1, column]}"
            data.at[index, column] = np.nan

data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4817 entries, 0 to 4816
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   arabic-uthmanic  4817 non-null   object
 1   arabic-imlaei    4817 non-null   object
 2   en-ahmedali      4817 non-null   object
 3   en-ahmedraza     4817 non-null   object
 4   en-arberry       4817 non-null   object
 5   en-asad          4817 non-null   object
 6   en-daryabadi     4817 non-null   object
 7   en-hilali        4817 non-null   object
 8   en-itani         4817 non-null   object
 9   en-maududi       4817 non-null   object
 10  en-mubarakpuri   4817 non-null   object
 11  en-pickthall     4817 non-null   object
 12  en-qarai         4817 non-null   object
 13  en-qaribullah    4817 non-null   object
 14  en-sahih         4817 non-null   object
 15  en-sarwar        4817 non-null   object
 16  en-shakir        4817 non-null   object
 17  en-wahiduddi     4817 non-null   

None

In [198]:
for col in df.columns:
    print(data.loc[1234, col], end="\n\n")

ٱلْأَعْرَابُ أَشَدُّ كُفْرًا وَنِفَاقًا وَأَجْدَرُ أَلَّا يَعْلَمُوا۟ حُدُودَ مَآ أَنزَلَ ٱللَّهُ عَلَىٰ رَسُولِهِۦ وَٱللَّهُ عَلِيمٌ حَكِيمٌ

الأَعرابُ أَشَدُّ كُفرًا وَنِفاقًا وَأَجدَرُ أَلّا يَعلَموا حُدودَ ما أَنزَلَ اللَّهُ عَلىٰ رَسولِهِ وَاللَّهُ عَليمٌ حَكيمٌ

The village Arabs are more obstinate in disbelief and hypocrisy, and impervious to ordinances revealed to His Apostle by God; yet God is aware of everything and is wise.

The ignorant are more severe in disbelief and hypocrisy, and deserve to remain ignorant of the commands which Allah has revealed to His Noble Messenger; and Allah is All Knowing, Wise.

The Bedouins are more stubborn in unbelief and hypocrisy, and apter not to know the bounds of what God has sent down on His Messenger; and God is All-knowing, All-wise.

 the bedouin are more tenacious in  refusal to acknowledge the truth and in  hypocrisy , and more liable to ignore the ordinances which God has bestowed from on high upon His Apostle - but God is all-kno

In [199]:
# Due to size limits on translators
def create_shards(data):
    shard_size = 0
    shard_number = 1
    start_index = 0
    
    while start_index < len(data):

        end_index = start_index
        while end_index < len(data):
            shard_size = shard_number * 1500

            if end_index > shard_size:
                break
            
            end_index += 1

        shard = data.iloc[start_index:end_index]
        shard.to_excel(f"./shards/shard_no_parantheses_{shard_number}.xlsx", index=False)

        start_index = end_index
        shard_number += 1

create_shards(data.iloc[:, 2:])


print("Go to https://translate.google.com/?sl=en&tl=ar&op=docs and translate the file, then put it in google drive")
print("If there is more translation services, APPLY THEM, The amount of potential data is huge")

Go to https://translate.google.com/?sl=en&tl=ar&op=docs and translate the file, then put it in google drive
If there is more translation services, APPLY THEM, The amount of potential data is huge


In [201]:
translated = pd.concat([pd.read_excel("./translated_v2/" + file) for file in os.listdir('./translated_v2/')]).reset_index(drop=True)

In [219]:
arabic_data = pd.concat([data.iloc[:, 1].to_frame(), translated], axis=1)

In [220]:
def prepare_text(text):
    
    if pd.isna(text):
        return np.nan
    
    # Convert text to lowercase
    text = text.lower()
    
    # Remove text between parentheses (Explanatory text which is not originally written in arabic)
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'\[[^]]*\)', '', text)
    
    # Remove dashes and commas
    text = re.sub(r'[\.!-,:;‘’“”\"\'\?]+', '', text)
    
    # Remove diacritics
    text = re.sub(r'[^\u0621-\u064A\s]+', '', text)
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    
    return text

arabic_data = arabic_data.map(prepare_text)

In [223]:
# Remove empty strings and starting ayat like "ألر"
arabic_data = arabic_data.map(lambda x: np.nan if
                              pd.isna(x) or
                              len(x) < 5 or
                              x.startswith('بسم الله الرحمن الرحيم') and len(x) < 30
                              else x)

arabic_data = arabic_data[arabic_data['arabic-imlaei'].notna()].reset_index(drop=True)

In [224]:
# Fill NaN values with neighbours
arabic_data = arabic_data.ffill(axis=1)

## The Cleaned Dataset

In [257]:
n = np.random.randint(0, 4795)

for col in arabic_data.columns:
    print(arabic_data.loc[n, col], end="\n\n")

إن المنافقين يخادعون الله وهو خادعهم وإذا قاموا إلى الصلاة قاموا كسالى يراءون الناس ولا يذكرون الله إلا قليلا مذبذبين بين ذلك لا إلى هؤلاء ولا إلى هؤلاء ومن يضلل الله فلن تجد له سبيلا

 إن المنافقين يحاولون خداع الله وهو يخدع أنفسهم وإذا قاموا إلى الصلاة فعلوها كسالى لا إلا رياء ولا يذكرون الله إلا قليلا مترددين بين ذلك لا مع هؤلاء ولا مع هؤلاء ومن لم يهديه الله فلن تجد له سبيلا

ولا شك أن المنافقون في أهوائهم يريدون خداع الله فيطفئهم ويغفلهم وإذا قاموا إلى الصلاة فعلوها كرها ورؤية للناس ولا يذكرون الله إلا قليلا متقلبة في الوسط لا هنا ولا هناك ومن يضلل الله فلن تجد له سبيلا

 إن المنافقين يريدون خداع الله والله يخدعهم وإذا قاموا إلى الصلاة قاموا كسالى يراءون الناس ولا يذكرون الله إلا قليلا مترددين طوال الوقت ليس لهؤلاء وليس لهؤلاء ومن يضلل الله فلن تجد له سبيلا

 إن المنافقين يخادعون الله وهو يخدعهم وإذا قاموا إلى الصلاة قاموا وهم وهم يراءون الناس ويحمدونهم ولا يذكرون الله إلا نادرا يترددون بين هذا وذاك لا إلى هؤلاء ولا إلى هؤلاء ومن يضلل الله فلن تجد له سبيلا

إن المنافقين ليخدعون ال

In [258]:
arabic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4795 entries, 0 to 4794
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   arabic-imlaei  4795 non-null   object
 1   الأحمدلي       4795 non-null   object
 2    en-ahmedraza  4795 non-null   object
 3    en-arberry    4795 non-null   object
 4    أون الأسد     4795 non-null   object
 5    en-daryabadi  4795 non-null   object
 6    en-hilali     4795 non-null   object
 7    عيتاني        4795 non-null   object
 8    المودودي      4795 non-null   object
 9    مباركبوري     4795 non-null   object
 10   en-pickthall  4795 non-null   object
 11   en-qarai      4795 non-null   object
 12   en-قرب الله   4795 non-null   object
 13   صحيح          4795 non-null   object
 14   en-sarwar     4795 non-null   object
 15   شاكر          4795 non-null   object
 16   en-وحيدودي    4795 non-null   object
 17   en-yusufali   4795 non-null   object
dtypes: object(18)
memory usage: 

In [259]:
arabic_data.to_excel("./new_data.xlsx", index=False)